In [251]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display_html

In [222]:
NHLData = pd.read_csv("/content/NHLStandings.csv")

Teams = {"Atlantic" : ["Florida Panthers", "Boston Bruins", "Toronto Maple Leafs",
            "Tampa Bay Lightning", "Detroit Red Wings", "Buffalo Sabres",
            "Ottawa Senators", 	"Montreal Canadiens"],

        "Central" : ["Dallas Stars", "Winnipeg Jets", "Colorado Avalanche",
                    "Nashville Predators", "St. Louis Blues", "Minnesota Wild",
                "Arizona Coyotes", "Chicago Blackhawks"],

        "Metropolitan" : ["New York Rangers", "Carolina Hurricanes", "New York Islanders",
                    "Washington Capitals", "Pittsburgh Penguins", "Philadelphia Flyers",
                    "New Jersey Devils", "Columbus Blue Jackets"],

        "Pacific" : ["Vancouver Canucks", "Edmonton Oilers", "Vegas Golden Knights",
                    "Los Angeles Kings", "Calgary Flames", 	"Seattle Kraken",
                    "Anaheim Ducks", "San Jose Sharks"]}

def get_division(team):
    return list(filter(lambda x: team in Teams[x], Teams))[0]

In [223]:
splits = []
for i in range(len(NHLData)):
    splits.append( list(map(int, NHLData["Overall"][i].split("-"))))
    ot_wins = int(NHLData["Overtime"][i].split("-")[0]) + int(NHLData["Shootout"][i].split("-")[0])
    splits[-1].append(int(splits[-1][0]) - ot_wins)
    splits[-1].append(ot_wins)

my_data = pd.DataFrame(splits, columns = ["W", "RL", "OTL", "RW", "OTW"])

my_data["Team"] = NHLData["Team"]
my_data = my_data[["Team", "RW", "OTW", "RL", "OTL"]]

my_data["Pts"] = 2*(my_data["RW"] + my_data["OTW"]) + my_data["OTL"]
my_data["NewPts"] = 3*my_data["RW"] + 2*my_data["OTW"] + my_data["OTL"]
my_data["Division"] = [get_division( my_data["Team"][i] ) for i in range(len(my_data)) ]

In [255]:
Point_types = ["Pts", "NewPts"]

def get_division_standings(Division, Points = "Pts", Playoffs = True):

    division_data = my_data.loc[my_data["Division"] == Division].sort_values(
        by = [Points, "RW"], ascending = False).reset_index(drop=True)
    division_data.index += 1
    if Playoffs:
        return division_data.drop(columns = ["Division", Point_types[(Point_types.index(Points)+1)%2 ]])[0:4]
    return division_data.drop(columns = ["Division", Point_types[(Point_types.index(Points)+1)%2 ]])

def get_conference_standings(Conference, Points = "Pts", Playoffs = True):
    East, West = ["Atlantic", "Metropolitan"], ["Central", "Pacific"]
    if Conference == "East":
        conference_data = my_data.loc[my_data["Division"].isin(East)].sort_values(
            by = [Points, "RW"], ascending = False).reset_index(drop=True)
        conference_data.index += 1
        if Playoffs:
            return conference_data.drop(columns = [Point_types[(Point_types.index(Points)+1)%2 ]])[0:8]
        return conference_data.drop(columns = [Point_types[(Point_types.index(Points)+1)%2 ]])

    conference_data = my_data.loc[my_data["Division"].isin(West)].sort_values(
        by = [Points, "RW"], ascending = False).reset_index(drop=True)
    conference_data.index += 1
    if Playoffs:
        return conference_data.drop(columns = [Point_types[(Point_types.index(Points)+1)%2 ]])[0:8]
    return conference_data.drop(columns = [Point_types[(Point_types.index(Points)+1)%2 ]])

In [249]:
def compare_tables(df1, df2):
    df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption('Current Points System')
    df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption('Proposed Points System')

    display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

def compare_conference(Conference):
    df1 = get_conference_standings(Conference)
    df2 = get_conference_standings(Conference, Points = "NewPts")
    return compare_tables(df1, df2)

def compare_division(Division):
    df1 = get_division_standings(Division)
    df2 = get_division_standings(Division, Points = "NewPts")
    return compare_tables(df1, df2)

In [263]:
for conf in ["East", "West"]:
    compare_conference(conf)

,Team,RW,OTW,RL,OTL,Pts,Division
1,New York Rangers,43,12,23,4,114,Metropolitan
2,Carolina Hurricanes,44,8,23,7,111,Metropolitan
3,Florida Panthers,42,10,24,6,110,Atlantic
4,Boston Bruins,36,11,20,15,109,Atlantic
5,Toronto Maple Leafs,33,13,26,10,102,Atlantic
6,Tampa Bay Lightning,37,8,29,8,98,Atlantic
7,New York Islanders,29,10,27,16,94,Metropolitan
8,Washington Capitals,32,8,31,11,91,Metropolitan
,Team,RW,OTW,RL,OTL,NewPts,Division
1,New York Rangers,43,12,23,4,157,Metropolitan


,Team,RW,OTW,RL,OTL,Pts,Division
1,Dallas Stars,40,12,21,9,113,Central
2,Vancouver Canucks,44,6,22,9,109,Pacific
3,Winnipeg Jets,45,6,24,6,108,Central
4,Colorado Avalanche,41,8,25,7,105,Central
5,Edmonton Oilers,39,10,26,6,104,Pacific
6,Nashville Predators,38,9,30,5,99,Central
7,Vegas Golden Knights,34,11,28,8,98,Pacific
8,Los Angeles Kings,37,6,27,11,97,Pacific
,Team,RW,OTW,RL,OTL,NewPts,Division
1,Winnipeg Jets,45,6,24,6,153,Central


In [262]:
for key in list(Teams.keys()):
    compare_division(key)

,Team,RW,OTW,RL,OTL,Pts
1,Florida Panthers,42,10,24,6,110
2,Boston Bruins,36,11,20,15,109
3,Toronto Maple Leafs,33,13,26,10,102
4,Tampa Bay Lightning,37,8,29,8,98
,Team,RW,OTW,RL,OTL,NewPts
1,Florida Panthers,42,10,24,6,152
2,Boston Bruins,36,11,20,15,145
3,Tampa Bay Lightning,37,8,29,8,135
4,Toronto Maple Leafs,33,13,26,10,135


,Team,RW,OTW,RL,OTL,Pts
1,Dallas Stars,40,12,21,9,113
2,Winnipeg Jets,45,6,24,6,108
3,Colorado Avalanche,41,8,25,7,105
4,Nashville Predators,38,9,30,5,99
,Team,RW,OTW,RL,OTL,NewPts
1,Winnipeg Jets,45,6,24,6,153
2,Dallas Stars,40,12,21,9,153
3,Colorado Avalanche,41,8,25,7,146
4,Nashville Predators,38,9,30,5,137


,Team,RW,OTW,RL,OTL,Pts
1,New York Rangers,43,12,23,4,114
2,Carolina Hurricanes,44,8,23,7,111
3,New York Islanders,29,10,27,16,94
4,Washington Capitals,32,8,31,11,91
,Team,RW,OTW,RL,OTL,NewPts
1,New York Rangers,43,12,23,4,157
2,Carolina Hurricanes,44,8,23,7,155
3,Washington Capitals,32,8,31,11,123
4,New York Islanders,29,10,27,16,123


,Team,RW,OTW,RL,OTL,Pts
1,Vancouver Canucks,44,6,22,9,109
2,Edmonton Oilers,39,10,26,6,104
3,Vegas Golden Knights,34,11,28,8,98
4,Los Angeles Kings,37,6,27,11,97
,Team,RW,OTW,RL,OTL,NewPts
1,Vancouver Canucks,44,6,22,9,153
2,Edmonton Oilers,39,10,26,6,143
3,Los Angeles Kings,37,6,27,11,134
4,Vegas Golden Knights,34,11,28,8,132
